In [98]:
import pandas as pd
import tensortrade.env.default as default
from tensortrade.agents import DQNAgent
from tensortrade.env.default.actions import BSH
from tensortrade.env.default.rewards import RiskAdjustedReturns, PBR
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.instruments import EUR, Instrument
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.wallets import Wallet, Portfolio
from agent.DQN import DQN
import ta
import itertools
%matplotlib inline

In [94]:
symbol = 'ALO'
ink = 'alstom'
inst = Instrument(symbol, 2, 'Mcphy energy stock')

data = pd.read_csv('../data/alstom_1h_2y.csv')
data = ta.add_all_ta_features(data, 'Open', 'High', 'Low', 'Close', 'Volume', fillna=True)
data.columns = [symbol + ":" + name.lower() for name in data.columns]

feed = DataFeed([
    Stream.source(list(data[col]), dtype="float") for col in data.columns if col != f'{symbol}:date'
])
feed.compile()

exchange = Exchange(ink, service=execute_order)(
    Stream.source(list(data[f'{symbol}:close'])).rename(f"EUR-{symbol}")
)

cash = Wallet(exchange, 3_000 * EUR)
asset = Wallet(exchange, 0 * inst)

portfolio = Portfolio(EUR, [
    cash,
    asset
])

renderer_feed = DataFeed([
    Stream.source(list(data["ALO:date"])).rename("date"),
    Stream.source(list(data["ALO:open"]), dtype="float").rename("open"),
    Stream.source(list(data["ALO:high"]), dtype="float").rename("high"),
    Stream.source(list(data["ALO:low"]), dtype="float").rename("low"),
    Stream.source(list(data["ALO:close"]), dtype="float").rename("close"),
    Stream.source(list(data["ALO:volume"]), dtype="float").rename("volume")
])
reward_scheme = RiskAdjustedReturns()
action_scheme = BSH(cash, asset)

env = default.create(
    portfolio=portfolio,
    action_scheme=action_scheme,
    reward_scheme=reward_scheme,
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)
env.reset()

/home/ludel/Workspace/IA/autotrading/venv/lib64/python3.6/site-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/home/ludel/Workspace/IA/autotrading/venv/lib64/python3.6/site-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



array([[  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       ...,
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       [ 35.32    ,  35.59    ,  34.9     , ..., -14.309275,   0.      ,
          0.      ]], dtype=float32)

In [87]:
data["ALO:open"].isna().value_counts()

False    4540
Name: ALO:open, dtype: int64

In [ ]:
episodes = [3,5,20,50,200,500]
steps = [50,100,200,500]
for n_steps in steps:
    for n_episodes in episodes:
        env.reset()
        agent = DQN(env)
        agent.train(n_steps=n_steps, n_episodes=n_episodes, render_interval=999999)
        agent.save('../save/')
        print(agent.id,n_steps,n_episodes, portfolio.net_worth)

== EPISODE ID (7/20): ba88b4b5-120a-4c40-a5d3-10e6568938f1 ==


In [ ]:
agent = DQN(env)
agent.restore('../save/savepolicy_network__b83a2652-2350-4407-a966-fc629874e0bb.hdf5')

In [97]:
print('Net worth {}'.format(portfolio.net_worth))

observation = env.reset()
print('Net worth {}'.format(portfolio.net_worth))

while True:
    action = agent.get_action(observation)
    observation, reward, done, info = env.step(action)
    if done:
        break
env.render()


Net worth 3000.0


FigureWidget({
    'data': [{'close': array([35.47999954, 36.09000015, 35.77000046, ..., 46.02000046, 46.16999…